In [1]:
import os
import numpy as np
from niceview.utils.mask import sparse_npz_to_array
from niceview.utils.tools import txt_to_list, select_col_from_name
import pandas as pd
import skimage
from sklearn.neighbors import NearestNeighbors
import scipy

In [2]:
DATA_PATH = '../data/'

In [3]:
def get_nuclei_pixels(cm, ad_cell_pos):
    """Get nuclei pixels.
    
    Args:
        cm (np.ndarray): Cell mask.
        ad_cell_pos (np.ndarray): Adherent cell positions.
    
    Returns:
        list: Nuclei region pixels.
    """
    tol = 1e-6
    regions = skimage.measure.regionprops(cm)
    xy = np.array([r.centroid[::-1] for r in regions])
    nbrs = NearestNeighbors(n_neighbors=1).fit(xy)
    distance, indices = nbrs.kneighbors(ad_cell_pos)
    c_index = indices[np.where(distance <= tol)]
    nuclei_region_pixels = [
        (regions[n].coords[:, 0], regions[n].coords[:, 1]) for n in c_index
    ]
    return nuclei_region_pixels


def paint_regions(image_shape, matched_regions, cell_colors_list):
    """Paint regions.
    
    Args:
        image_shape (tuple): Image shape.
        matched_regions (list): Matched regions.
        cell_colors_list (list): Cell colors list.
    
    Returns:
        np.ndarray: Filled image.
    """
    filled = np.ma.masked_all(image_shape)
    for i, r in enumerate(matched_regions):
        cc, rr = r
        filled[cc, rr] = cell_colors_list[i]
    return filled

In [4]:
# read data
mask = sparse_npz_to_array(os.path.join(DATA_PATH, 'gt-iz-p9-rep2-mask.npz'))
cell_pos = pd.read_csv(DATA_PATH + 'gt-iz-p9-rep2-cells-info.csv')[['x', 'y']].values
cells_gene = sparse_npz_to_array(os.path.join(DATA_PATH, 'gt-iz-p9-rep2-cells-gene.npz'))
gene_names = txt_to_list(os.path.join(DATA_PATH, 'gt-iz-p9-rep2-cells-gene-names.txt'))

In [5]:
# select gene
# ENSG00000065534
# ENSG00000037280
gene = select_col_from_name(cells_gene, gene_names, 'ENSG00000065534')

In [6]:
# find nuclei pixels
matched_regions = get_nuclei_pixels(mask, cell_pos)

In [7]:
# paint regions
mask_filtered_relabeled = paint_regions(mask.shape, matched_regions, cell_colors_list=gene + 1e-6)
mask_filtered_relabeled = scipy.sparse.csr_matrix(mask_filtered_relabeled.data)

In [8]:
# save mask
scipy.sparse.save_npz(DATA_PATH + 'gt-iz-p9-rep2-mask-filtered-relabeled.npz', mask_filtered_relabeled)